In [1]:
#Modulo de llamadas http para descargar ficheros
!pip install requests

#Libreria del problema TSP: http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsplib.html
!pip install tsplib95

In [3]:
import tsplib95
from urllib import request

In [6]:
# DATOS DEL PROBLEMA
file = "swiss42.tsp"

request.urlretrieve(
    "http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz",
    file + ".gz",
)
# !gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos
problem = tsplib95.load(file)

# Nodos
Nodos = list(problem.get_nodes())


# Devuelve la distancia entre dos nodos
def distancia(a, b, problem):
    return problem.get_weight(a, b)


# Devuelve la distancia total de una trayectoria/solucion(lista de nodos)
def distancia_total(solucion, problem):
    distancia_total = 0
    for i in range(len(solucion) - 1):
        distancia_total += distancia(solucion[i], solucion[i + 1], problem)
    return distancia_total + distancia(
        solucion[len(solucion) - 1], solucion[0], problem
    )

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41]


##Algoritmo de colonia de hormigas

La función Add_Nodo selecciona al azar un nodo con probabilidad uniforme.
Para ser mas eficiente debería seleccionar el próximo nodo siguiendo la probabilidad correspondiente a la ecuación:

$p^k_{ij}(t) = \frac{[\tau_{ij}(t)]^\alpha[\nu_{ij}]^\beta}{\sum_{l\in J^k_i} [\tau_{il}(t)]^\alpha[\nu_{il}]^\beta}$, si $j \in J^k_i$

$p^k_{ij}(t) = 0$, si $j \notin J^k_i$


In [46]:
alpha = 1.0
beta = 2.0


def Add_Nodo(problem, H, T):
    # Mejora:Establecer una funcion de probabilidad para
    # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
    not_visited_nodes = list(set(problem.get_nodes()) - set(H))

    curr_node = H[-1]
    total = 0
    numerador = {}
    for next_node in not_visited_nodes:
        dist = distancia(curr_node, next_node, problem)
        numerador[next_node] = (T[curr_node][next_node] ** alpha) * ((1 / dist) ** beta)
        total += numerador[next_node]

    prob = {}
    for nodo in not_visited_nodes:
        prob[nodo] = numerador[nodo] / total

    return max(prob, key=lambda x: prob[x])


def Incrementa_Feromona(problem, T, H):
    # Incrementa segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total
    for i in range(len(H) - 1):
        T[H[i]][H[i + 1]] += 1000 / distancia_total(H, problem)
    return T


def Evaporar_Feromonas(T):
    # Evapora 0.3 el valor de la feromona, sin que baje de 1
    # Mejora:Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
    T = [[max(T[i][j] - 0.3, 1) for i in range(len(Nodos))] for j in range(len(Nodos))]
    return T


def hormigas(problem, N: int):
    # Nodos
    Nodos = list(problem.get_nodes())

    # Inicializa las aristas con una cantidad inicial de feromonas:1
    # Mejora: inicializar con valores diferentes dependiendo diferentes criterios
    T = [[1 for _ in range(len(Nodos))] for _ in range(len(Nodos))]

    # Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
    Hormiga = [[0] for _ in range(N)]

    # Recorre cada agente construyendo la solución
    for h in range(len(Hormiga)):

        # Para cada agente se construye un camino
        for i in range(len(Nodos) - 1):
            # Elige el siguiente nodo
            Nuevo_Nodo = Add_Nodo(problem, Hormiga[h], T)
            Hormiga[h].append(Nuevo_Nodo)

        # Incrementa feromonas en esa arista
        T = Incrementa_Feromona(problem, T, Hormiga[h])
        # print("Feromonas(1)", T)

        # Evapora Feromonas
        T = Evaporar_Feromonas(T)
        # print("Feromonas(2)", T)

        # Seleccionamos el mejor agente

    mejor_solucion = []
    mejor_distancia = 10e100
    for h in range(N):
        distancia_actual = distancia_total(Hormiga[h], problem)
        if distancia_actual < mejor_distancia:
            mejor_solucion = Hormiga[h]
            mejor_distancia = distancia_actual

    print(mejor_solucion)
    print(mejor_distancia)


hormigas(problem, 10)

[0, 1, 6, 4, 3, 2, 27, 28, 29, 30, 32, 34, 20, 33, 31, 17, 37, 15, 14, 16, 19, 13, 5, 26, 18, 12, 11, 25, 10, 8, 9, 23, 41, 21, 39, 22, 38, 24, 40, 7, 36, 35]
1630
